In [2]:
# !pip install ratsnlp

## 말뭉치 내려받기 및 전처리
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 24.2MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 12.9MB/s]                            


In [12]:
## NSMC 전처리

import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')
write_lines("content/train.txt", nsmc.train.get_all_texts())
write_lines("content/test.txt", nsmc.test.get_all_texts())

In [14]:
## 디렉터리 만들기
import os
os.makedirs('nlpbook/bbpe', exist_ok=True)

In [15]:
## 바이트 수준 BPE 어휘 집합 구축

from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["content/train.txt", "content/test.txt"],
    vocab_size=10000,
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("nlpbook/bbpe")

['nlpbook/bbpe\\vocab.json', 'nlpbook/bbpe\\merges.txt']

In [16]:
## 워드피스 디렉터리 만들기
import os
os.makedirs("nlpbook/wordpiece", exist_ok=True)

In [17]:
## 워드피스 어휘 집합 구축
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["content/train.txt", "content/test.txt"],
    vocab_size=10000,
)
wordpiece_tokenizer.save_model("nlpbook/wordpiece")

['nlpbook/wordpiece\\vocab.txt']

In [18]:
## GPT 토크나이저 선언
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file nlpbook/bbpe\config.json not found


In [21]:
## GPT 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]

tokenizer_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

In [22]:
tokenizer_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

In [24]:
## GPT 모델 입력 만들기
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",  # 문장의 최대 길이에 맞춰 패딩
    max_length=12,  # 문장의 토근 기준 최대 길이
    truncation=True  # 문장 잘림 옵션 허용
)

In [25]:
batch_inputs["input_ids"]

[[335, 2339, 264, 582, 4056, 465, 3809, 0, 0, 0, 0, 0],
 [3694, 337, 2877, 759, 2884, 357, 807, 423, 9876, 876, 2961, 7293],
 [4958, 452, 3654, 264, 0, 0, 0, 0, 0, 0, 0, 0]]

In [26]:
batch_inputs["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [27]:
## BERT 토크나이저 선언
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "nlpbook/wordpiece",
    do_lower_case=False,
)

file nlpbook/wordpiece\config.json not found


In [28]:
## BERT 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]

tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [29]:
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

In [30]:
## BERT 모델 입력 만들기
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True
)

In [31]:
batch_inputs["input_ids"]

[[2, 620, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1344, 16, 3],
 [2, 3274, 9509, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [32]:
batch_inputs["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [33]:
batch_inputs["token_type_ids"]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]